<a href="https://colab.research.google.com/github/debanandasen/Tensorflow/blob/main/04_transfer_learning_in_temsorflow_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Transfer Learning with Tensorflow Part 1: Feature Extraction

There are two main benefits to using transfer learning:

1) Can leverage an existing neural network architecture proven to work on
   problems similar to our own.
   
2) Can leverage a working neural network architecture which has already learned
   patterns on similar data to our own. This often results in achieving great results with less custom data.

In [ ]:
# Add timestamp
import datetime
print(f"Notebook last run (end-to-end): {datetime.datetime.now()}")


# Are we running a GPU
!nvidia-smi

## Transfer Learning with Tensorflow HUb: with 10% of the data

In [ ]:
# Download and becoming one with Data
import zipfile
# Downlaod Data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# unzip the downloaded file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

In [ ]:
# How many images in each folder ?

import os

# Walk through 10 percent data dictionary and list number of files
for dirpath,dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} impages in '{dirpath}'.")


## Create data loaders (Preparing the data)

In [ ]:
# Setup data inputs
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224,224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training Images:")

train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                          target_size=IMAGE_SHAPE,
                                                          batch_size=BATCH_SIZE,
                                                          class_mode="categorical")

print("Testing Images")

test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=IMAGE_SHAPE,
                                             batch_size=BATCH_SIZE,
                                             class_mode="categorical")





## Setting up callbacks( things to run whilst our model trains)

In [ ]:
from tensorflow.python.ops.gen_linalg_ops import log_matrix_determinant_eager_fallback
# Create TensorBoard callback
import datetime

def create_tensorboard_callback(dir_name,experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir =log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

## Creating model using tensorflow Hub
Feature vector model link
https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1


In [ ]:
# Let's compare the following tow model
resnet_url ="https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"

efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [ ]:
# Import dependencies
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers


In [ ]:
#Let's make a create_model() function to create a model frol url
def create_model(model_url, num_classes=10):
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False,
                                           name="feature_extraction_layer",
                                           input_shape=IMAGE_SHAPE+(3,))

  # Create our own model
  model = tf.keras.Sequential([
      feature_extractor_layer,
      layers.Dense(num_classes, activation="softmax", name = "output_layer")
  ])

  return model

## creating and testing ResNet Tensorflow  Hub Feature Extraction model

In [ ]:
# Create Resnet Model
resnet_model = create_model(resnet_url,
                            num_classes=train_data_10_percent.num_classes)

In [ ]:
num_classes=train_data_10_percent.num_classes

In [ ]:
# Compile our resnet model

resnet_model.compile(loss="categorical_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

In [ ]:
resnet_model.summary()

In [ ]:
# Let's fit our ResNet model to the data

resnet_history = resnet_model.fit(train_data_10_percent,
                                  epochs=5,
                                  steps_per_epoch=len(train_data_10_percent),
                                  validation_data=test_data,
                                  validation_steps=len(test_data),
                                  callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub",
                                                                         experiment_name="resnet50v2")])



## Loss Curves

In [ ]:
import matplotlib.pyplot as plt

# Plot the validation and training curves
def plot_loss_curves(history):
 loss = history.history["loss"]
 val_loss = history.history["val_loss"]

 accuracy = history.history["accuracy"]
 val_accuracy = history.history["val_accuracy"]

 epochs = range(len(history.history["loss"]))

 # Plot loss
 plt.figure()
 plt.plot(epochs, loss, label="training_loss")
 plt.plot(epochs, val_loss, label="val_loss")
 plt.title("Loss")
 plt.xlabel("Epochs")
 plt.legend()

 # Plot accuracy
 plt.figure()
 plt.plot(epochs, accuracy, label="training_accuracy")
 plt.plot(epochs, val_accuracy, label="val_accuracy")
 plt.title("Accuracy")
 plt.xlabel("Epochs")
 plt.legend();

In [ ]:
plot_loss_curves(resnet_history)

## Creating and testing EfficentNetB0 Tensorflow Hub Feature Extraction Model

In [ ]:
# Creating EfficentNet Feature Extractor Model
efficientnet_model = create_model(model_url=efficientnet_url,
                                  num_classes=train_data_10_percent.num_classes)

# Compile The EfficientNet Model
efficientnet_model.compile(loss="categorical_crossentropy",
                        optimizer=tf.keras.optimizers.Adam(),
                         metrics=["accuracy"])

# Train Model with 10% of Training Data

efficientnet_history = efficientnet_model.fit(train_data_10_percent,
                                              epochs=5,
                                              steps_per_epoch=len(train_data_10_percent),
                                              validation_data=test_data,
                                              validation_steps=len(test_data),
                                              callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub",
                                                                         experiment_name="efficientnetb0")])

In [ ]:
plot_loss_curves(efficientnet_history)

In [ ]:
efficientnet_model.summary()

In [ ]:
resnet_model.summary()

## Comparing our model results using TensorBoard
  🔑 Note: When you upload things to tensorBoad.dev , your experiments are public.


In [ ]:
#View your TensorBoard at https://tensorboard.dev/experiment/RKFX2V4BTV6RUYk9CEIORQ/
!tensorboard dev list